In [218]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
nltk.download('wordnet')
import pandas_profiling as pp
from markupsafe import escape
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split, cross_val_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import random

# model building imports
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.layers import Conv1D, SimpleRNN, Bidirectional, MaxPooling1D, GlobalMaxPool1D, LSTM, GRU
from keras.models import Sequential
from keras.regularizers import L1L2

%matplotlib inline

# matplotlib defaults
plt.style.use("ggplot")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

import warnings 
warnings.filterwarnings('ignore')
nltk.download('omw-1.4')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [34]:
df = pd.read_json('/content/drive/MyDrive/News_Category_Dataset_v3.json', lines=True)
df.head()
df = df.iloc[:10000]

In [35]:
# shape of the dataset
print(df.shape)
# total number of unique categories
print("Unique categories:",df['category'].nunique())
print("-------------------------------------------------")
# information about metadata
df.info()

(10000, 6)
Unique categories: 29
-------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   link               10000 non-null  object        
 1   headline           10000 non-null  object        
 2   category           10000 non-null  object        
 3   short_description  10000 non-null  object        
 4   authors            10000 non-null  object        
 5   date               10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 468.9+ KB


In [36]:
# general descrption of dataset
df.describe().style.set_properties(**{'background-color': '#F0F0F0',
                                    'color': '#222222',
                                    'border': '1.5px  solid black'})

,link,headline,category,short_description,authors,date
count,10000,10000,10000,10000,10000,10000
unique,10000,9967,29,9995,1652,1607
top,https://www.huffpost.com/entry/covid-boosters-uptake-us_n_632d719ee4b087fae6feaac9,What To Watch On Amazon Prime That’s New This Week,POLITICS,The 25 most profound “Shower Thoughts” on Reddit from the last week.,,2018-05-03 00:00:00
freq,1,13,3361,3,821,75
first,nan,nan,nan,nan,nan,2018-05-01 00:00:00
last,nan,nan,nan,nan,nan,2022-09-23 00:00:00


In [37]:
# drop columns like authors, links and date as they are irrelevent to our problem.
new_df = df.drop(columns=['authors','link','date'])
new_df.head()

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...


In [38]:
# Top categories by and number of articles per categories
cat_df = pd.DataFrame(new_df['category'].value_counts()).reset_index()
cat_df.rename(columns={'index':'news_classes','category':'numcat'}, inplace=True)

In [39]:
# create final dataframe of combined headline and short_description
final_df = new_df.copy()
final_df['length_of_news'] = final_df['headline'] + final_df['short_description']
final_df.drop(['headline','short_description'], inplace=True, axis=1)
final_df['len_news'] = final_df['length_of_news'].map(lambda x: len(x))
final_df.head()

,category,length_of_news,len_news
0,U.S. NEWS,Over 4 Million Americans Roll Up Sleeves For O...,230
1,U.S. NEWS,"American Airlines Flyer Charged, Banned For Li...",248
2,COMEDY,23 Of The Funniest Tweets About Cats And Dogs ...,133
3,PARENTING,The Funniest Tweets From Parents This Week (Se...,215
4,U.S. NEWS,Woman Who Called Cops On Black Bird-Watcher Lo...,233


In [96]:
# maximum length of news in each category
lenmax_df = final_df.groupby('category')['len_news'].max().reset_index().sort_values(by='len_news',ascending=False)
lenmax_df.head(42)

,category,len_news
15,POLITICS,312
28,WORLD NEWS,296
23,U.S. NEWS,287
5,EDUCATION,278
12,MEDIA,275
0,BLACK VOICES,265
1,BUSINESS,263
4,CULTURE & ARTS,262
6,ENTERTAINMENT,258
14,PARENTING,258


In [41]:
# wordcloud of categories of news articles in our dataset
plt.figure(figsize=(12,12))
wc = WordCloud(max_words=1000, 
               min_font_size=10,
               height=600,
               width=1600,
               background_color='black',
               contour_color='black',
               colormap='plasma',
               repeat=False,
               stopwords=STOPWORDS).generate(' '.join(final_df.category))

<Figure size 864x864 with 0 Axes>

In [42]:
# create new dataframe of category and length of each news articles in that categories
ndf = final_df.copy()
ndf.drop('len_news', inplace=True, axis=1)

# list of top 10  categories in out dataset
categories = cat_df['news_classes'][:10].to_list()

# list of news articles of each top 10 categories list
articles_list = []

for i in categories:
    cat_ndf = ndf[ndf['category'] == i]
    cat_array = cat_ndf['length_of_news'].values  # array of news articles text in each category
    articles_list.append(cat_array)
    
# create a wordcloud instance
wc1 = WordCloud(max_words=1000, 
               min_font_size=10,
               height=600,
               width=1600,
               background_color='black',
               contour_color='black',
               colormap='plasma',
               repeat=True,
               stopwords=STOPWORDS)

In [43]:
# start with removing some data from politics dataframe ndf
politics_list = list(ndf[ndf['category'] == 'POLITICS'].index) 
list_16 = politics_list[:16000] # list of 16000 row labels of POLITICS category

# drop these 16000 labels from the dataset
ndf2 = ndf.copy()
ndf2.drop(list_16, axis=0, inplace=True)

In [44]:
# clean the text data using regex and data cleaning function
def datacleaning(text):
    whitespace = re.compile(r"\s+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    text = whitespace.sub(' ', text)
    text = user.sub('', text)
    text = re.sub(r"\[[^()]*\]","", text)
    text = re.sub("\d+", "", text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
    text = text.lower()
    
    # removing stop-words
    text = [word for word in text.split() if word not in list(STOPWORDS)]
    
    # word lemmatization
    sentence = []
    for word in text:
        lemmatizer = WordNetLemmatizer()
        sentence.append(lemmatizer.lemmatize(word,'v'))
        
    return ' '.join(sentence) 

In [45]:
# Example of pre-processing using above function
print("Text sentence before pre-processing:\n",ndf2['length_of_news'][0])
print("---"*35)
print("Text sentence after pre-processing:\n",datacleaning(ndf2['length_of_news'][0]))

Text sentence before pre-processing:
 Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID BoostersHealth experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.
---------------------------------------------------------------------------------------------------------
Text sentence after pre-processing:
 million americans roll sleeves omicrontargeted covid boostershealth experts say early predict whether demand match million dose new boosters us order fall


In [46]:
# apply datacleaning function to column 'length_of_news'
ndf2['length_of_news'] = ndf2['length_of_news'].apply(datacleaning)

In [47]:
# tokenization and vectorization workflow
# word level one-hot encoding for sample data

samples = list(ndf2['length_of_news'][:5].values)  # samples of first 5 documents of out dataset

token_index = {}  # builds an index of tokens in the data
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1 # assigning unique index to each unique words
            
max_length = 15

results = np.zeros(shape=(len(samples),   # results will be stored in this array
                          max_length,
                          max(token_index.values()) +1)) 

print("Shape of stored results array:", results.shape)
print("Token index of unique words: \n", token_index)

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i,j,index] = 1

Shape of stored results array: (5, 15, 91)
Token index of unique words: 
 {'million': 1, 'americans': 2, 'roll': 3, 'sleeves': 4, 'omicrontargeted': 5, 'covid': 6, 'boostershealth': 7, 'experts': 8, 'say': 9, 'early': 10, 'predict': 11, 'whether': 12, 'demand': 13, 'match': 14, 'dose': 15, 'new': 16, 'boosters': 17, 'us': 18, 'order': 19, 'fall': 20, 'american': 21, 'airlines': 22, 'flyer': 23, 'charge': 24, 'ban': 25, 'life': 26, 'punch': 27, 'flight': 28, 'attendant': 29, 'videohe': 30, 'subdue': 31, 'passengers': 32, 'crew': 33, 'flee': 34, 'back': 35, 'aircraft': 36, 'confrontation': 37, 'accord': 38, 'attorneys': 39, 'office': 40, 'los': 41, 'angeles': 42, 'funniest': 43, 'tweet': 44, 'cat': 45, 'dog': 46, 'week': 47, 'sept': 48, 'dont': 49, 'understand': 50, 'eat': 51, 'parent': 52, 'accidentally': 53, 'put': 54, 'grownup': 55, 'toothpaste': 56, 'toddlers': 57, 'toothbrush': 58, 'scream': 59, 'clean': 60, 'teeth': 61, 'carolina': 62, 'reaper': 63, 'dip': 64, 'tabasco': 65, 'sauce

In [48]:
# one hot encoding using keras tokenizer and pad sequencing
X = ndf2['length_of_news']
encoder = LabelEncoder()
y = encoder.fit_transform(ndf2['category'])
print("shape of input data: ", X.shape)
print("shape of target variable: ", y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

tokenizer = Tokenizer(num_words=100000, oov_token='<00V>') 
tokenizer.fit_on_texts(X_train) # build the word index
# padding X_train text input data
train_seq = tokenizer.texts_to_sequences(X_train) # converts strinfs into integer lists
train_padseq = pad_sequences(train_seq, maxlen=130) # pads the integer lists to 2D integer tensor 

# padding X_test text input data
test_seq = tokenizer.texts_to_sequences(X_test)
test_padseq = pad_sequences(test_seq, maxlen=130)

word_index = tokenizer.word_index
max_words = 150000  # total number of words to consider in embedding layer
total_words = len(word_index)
maxlen = 130 # max length of sequence 
y_train = to_categorical(y_train, num_classes=42)
y_test = to_categorical(y_test, num_classes=42)
print("Length of word index:", total_words)

shape of input data:  (6639,)
shape of target variable:  (6639,)
Length of word index: 18799


In [301]:
cat_enc_arry = np.unique(y)
cat_arry = np.unique(ndf2['category'])

In [49]:
# basline model using embedding layers and simpleRNN
model = Sequential()
model.add(Embedding(total_words, 70, input_length=maxlen))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1, recurrent_dropout=0.20, activation='tanh', return_sequences=True)))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1, recurrent_dropout=0.30, activation='tanh', return_sequences=True)))
model.add(SimpleRNN(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(42, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 130, 70)           1315930   
                                                                 
 bidirectional_6 (Bidirectio  (None, 130, 128)         17280     
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 130, 128)         24704     
 nal)                                                            
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 32)                5152      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 42)               

In [50]:
model.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy']
            )
# SETUP A EARLY STOPPING CALL and model check point API
earlystopping = keras.callbacks.EarlyStopping(monitor='accuracy',
                                             patience=5,
                                              verbose=1,
                                              mode='min'
                                             )
checkpointer = ModelCheckpoint(filepath='bestvalue',moniter='val_loss', verbose=0, save_best_only=True)
callback_list = [checkpointer, earlystopping]

# fit model to the data
history = model.fit(train_padseq, y_train, 
                   batch_size=128, 
                    epochs=15, 
                    validation_split=0.2
                   )

# evalute the model
test_loss, test_acc = model.evaluate(test_padseq, y_test, verbose=0)
print("test loss and accuracy:", test_loss, test_acc)

Epoch 1/15
34/34 [==============================] - 33s 833ms/step - loss: 3.1702 - accuracy: 0.1787 - val_loss: 2.6858 - val_accuracy: 0.2117
Epoch 2/15
34/34 [==============================] - 27s 802ms/step - loss: 2.6045 - accuracy: 0.2328 - val_loss: 2.5634 - val_accuracy: 0.2117
Epoch 3/15
34/34 [==============================] - 26s 781ms/step - loss: 2.5455 - accuracy: 0.2340 - val_loss: 2.5448 - val_accuracy: 0.2117
Epoch 4/15
34/34 [==============================] - 27s 795ms/step - loss: 2.5136 - accuracy: 0.2432 - val_loss: 2.5301 - val_accuracy: 0.2117
Epoch 5/15
34/34 [==============================] - 25s 739ms/step - loss: 2.4186 - accuracy: 0.3067 - val_loss: 2.5614 - val_accuracy: 0.2352
Epoch 6/15
34/34 [==============================] - 29s 865ms/step - loss: 2.2438 - accuracy: 0.3865 - val_loss: 2.5226 - val_accuracy: 0.2615
Epoch 7/15
34/34 [==============================] - 26s 751ms/step - loss: 2.0941 - accuracy: 0.4468 - val_loss: 2.3586 - val_accuracy: 0.3481

In [395]:
# Twitter text processing:
test = ndf2['length_of_news'][17]
print(test)
test_seq = tokenizer.texts_to_sequences(test)
test_paseq = pad_sequences(test_seq, maxlen=130)
x = model.predict(test_paseq)
pd.DataFrame(x).head(10)

maury will basestealing shortstop dodgers die maury will help los angeles dodgers win three world series title basestealing prowess die
5/5 [==============================] - 0s 48ms/step


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.056841,0.004542,0.038531,0.035957,0.002997,0.000836,0.777580,0.003846,0.001150,0.004085,...,0.000138,0.000154,0.000119,0.000093,0.000116,0.000166,0.000088,0.000168,0.000078,0.000113
1,0.123902,0.004705,0.048620,0.092377,0.003098,0.001123,0.485794,0.008612,0.002323,0.002243,...,0.000181,0.000237,0.000190,0.000156,0.000151,0.000263,0.000146,0.000238,0.000140,0.000164
2,0.010992,0.001448,0.002785,0.008340,0.003149,0.000334,0.935172,0.001140,0.000447,0.006606,...,0.000122,0.000141,0.000089,0.000102,0.000066,0.000129,0.000054,0.000136,0.000106,0.000118
3,0.123902,0.004705,0.048620,0.092377,0.003098,0.001123,0.485794,0.008612,0.002323,0.002243,...,0.000181,0.000237,0.000190,0.000156,0.000151,0.000263,0.000146,0.000238,0.000140,0.000164
4,0.123902,0.004705,0.048620,0.092377,0.003098,0.001123,0.485794,0.008612,0.002323,0.002243,...,0.000181,0.000237,0.000190,0.000156,0.000151,0.000263,0.000146,0.000238,0.000140,0.000164
5,0.013742,0.001558,0.007528,0.009510,0.001621,0.000303,0.940467,0.000906,0.000301,0.002954,...,0.000086,0.000090,0.000081,0.000065,0.000064,0.000100,0.000050,0.000108,0.000063,0.000087
6,0.020252,0.039446,0.145505,0.012774,0.007706,0.004564,0.028327,0.048740,0.042582,0.076759,...,0.000135,0.000074,0.000096,0.000139,0.000122,0.000202,0.000052,0.000095,0.000057,0.000114
7,0.123902,0.004705,0.048620,0.092377,0.003098,0.001123,0.485794,0.008612,0.002323,0.002243,...,0.000181,0.000237,0.000190,0.000156,0.000151,0.000263,0.000146,0.000238,0.000140,0.000164
8,0.037684,0.003349,0.006366,0.042155,0.002816,0.000561,0.656507,0.007074,0.001749,0.006837,...,0.000127,0.000211,0.000197,0.000172,0.000094,0.000154,0.000095,0.000150,0.000180,0.000139
9,0.037684,0.003349,0.006366,0.042155,0.002816,0.000561,0.656507,0.007074,0.001749,0.006837,...,0.000127,0.000211,0.000197,0.000172,0.000094,0.000154,0.000095,0.000150,0.000180,0.000139


In [396]:
x2 = pd.DataFrame(x)
max(x2.iloc[0])
x3 = x2.iloc[0].tolist()
mx_indx = x3.index(max(x2.iloc[0]))
print(mx_indx)
category_fin = cat_arry[mx_indx]
print(category_fin)

6
ENTERTAINMENT
